In [1]:
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from scipy import signal
from pathlib import Path
import tensorflow_probability as tfp
import tensorflow as tf
from tqdm import tqdm  
from os import path
from myfunctions import event_separation , image_crop, image_crop_negative, poi



## Input data from a folder ##

In [2]:
files_dir = r'C:\Users\roumba\Documents\Software\deep-events\original_data'
images_dir = '221123_mtStayGold_U2OS_ZEISS_fl'
joined_path = os.path.join(files_dir, images_dir)
size=(2048,2048)
img,input_name,output_name,datacsv={},{},{},{}

for input_file in os.listdir(joined_path):
        joined_file_path = os.path.join(files_dir, images_dir, input_file)
        if '.tif' in input_file:
            date, cell_type, dye, bf_fl, index  = input_file.split('_')
            number, ome, tiff= index.split('.')
            img[int(number)-1] = Image.open(joined_file_path)
            input_name[int(number)-1]= f'points_{date}_{cell_type}_{bf_fl}_{number}'
            output_name[int(number)-1]= f'image_{date}_{cell_type}_{bf_fl}_{number}'
            print('Loaded image:'+input_file)
        else:
            labels, index, bf_fl  = input_file.split('_')
            datacsv[int(index)-1] = pd.read_csv(joined_file_path)
            print('Loaded csv:'+ input_file)
max_number=int(len(datacsv))   

Loaded image:221123_U2OS_mitostaygold_GFP_1.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_10.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_11.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_12.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_2.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_3.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_4.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_5.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_6.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_7.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_8.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_9.ome.tif
Loaded csv:labels_10_fl.csv
Loaded csv:labels_11_fl.csv
Loaded csv:labels_12_fl.csv
Loaded csv:labels_1_fl.csv
Loaded csv:labels_2_fl.csv
Loaded csv:labels_3_fl.csv
Loaded csv:labels_4_fl.csv
Loaded csv:labels_5_fl.csv
Loaded csv:labels_6_fl.csv
Loaded csv:labels_7_fl.csv
Loaded csv:labels_8_fl.csv
Loaded csv:labels_9_fl.csv


## Get all Gaussian Points of Interest ##

In [3]:
for x in range(max_number):
    csv=datacsv[x]
    im=img[x]
    for i in range(1,10,1):
        sigma=(i,i)
        s=sigma[0]
        in_name=f'{input_name[x]}_sigma{s}.tiff'
        framenum=im.n_frames
        poi(csv,in_name,sigma,size,framenum)

100%|██████████| 47/47 [00:12<00:00,  3.88it/s]


## Cut Images and Gaussian POIs ##

In [5]:
base_dir= r'C:\Users\roumba\Documents\Software\deep-events'
training_dir= r'C:\Users\roumba\Documents\Software\deep-events\training_data'
folder_name=images_dir+'_pos'
filepath= os.path.join(training_dir , folder_name)

for x in range(max_number):
    csv=datacsv[x]
    im=img[x]
    print('file number:',x)
    for i in range(1,10,1):
        sigma=(i,i)
        s=sigma[0]
        print('sigma:',s)
        in_name=f'{input_name[x]}_sigma{s}.tiff'
        out_name=f'{output_name[x]}'
        out_name_g=f'{output_name[x]}_sigma{s}'
    
        list1=event_separation(csv)
        l=len(list1)
        image_crop(l,list1, csv, im,0, out_name,filepath)

        gauss=base_dir+f'\{in_name}'
        gauss_image=Image.open(gauss)
        image_crop(l,list1, csv, gauss_image,1, out_name_g,filepath)

file number: 0
sigma: 1
52.0
50.0
22.0
52.0
50.0
22.0
sigma: 2
52.0
50.0
22.0
52.0
50.0
22.0
sigma: 3
52.0
50.0
22.0
52.0
50.0
22.0
sigma: 4
52.0
50.0
22.0
52.0
50.0
22.0
sigma: 5
52.0
50.0
22.0
52.0
50.0
22.0
sigma: 6
52.0
50.0
22.0
52.0
50.0
22.0
sigma: 7
52.0
50.0
22.0
52.0
50.0
22.0
sigma: 8
52.0
50.0
22.0
52.0
50.0
22.0
sigma: 9
52.0
50.0
22.0
52.0
50.0
22.0
file number: 1
sigma: 1
132.0
37.0
23.0
167.0
12.0
132.0
37.0
23.0
167.0
12.0
sigma: 2
132.0
37.0
23.0
167.0
12.0
132.0
37.0
23.0
167.0
12.0
sigma: 3
132.0
37.0
23.0
167.0
12.0
132.0
37.0
23.0
167.0
12.0
sigma: 4
132.0
37.0
23.0
167.0
12.0
132.0
37.0
23.0
167.0
12.0
sigma: 5
132.0
37.0
23.0
167.0
12.0
132.0
37.0
23.0
167.0
12.0
sigma: 6
132.0
37.0
23.0
167.0
12.0
132.0
37.0
23.0
167.0
12.0
sigma: 7
132.0
37.0
23.0
167.0
12.0
132.0
37.0
23.0
167.0
12.0
sigma: 8
132.0
37.0
23.0
167.0
12.0
132.0
37.0
23.0
167.0
12.0
sigma: 9
132.0
37.0
23.0
167.0
12.0
132.0
37.0
23.0
167.0
12.0
file number: 2
sigma: 1
30.0
12.0


c:\Users\roumba\Documents\Software\deep-events\.env\lib\site-packages\tifffile\tifffile.py:2001: UserWarning: <tifffile.TiffWriter 'image_221123_U2OS_GFP_3_0.tiff'> writing zero-size array to nonconformant TIFF
  warnings.warn(


205.0
19.0
63.0
252.0
254.0
18.0
80.0
227.0
30.0
12.0
205.0
19.0
63.0
252.0


c:\Users\roumba\Documents\Software\deep-events\.env\lib\site-packages\tifffile\tifffile.py:2001: UserWarning: <tifffile.TiffWriter 'image_221123_U2O…ma1_0gauss.tiff'> writing zero-size array to nonconformant TIFF
  warnings.warn(


254.0
18.0
80.0
227.0
sigma: 2
30.0
12.0
205.0
19.0
63.0
252.0
254.0
18.0
80.0
227.0
30.0


...

254.0
18.0
80.0
227.0
sigma: 8
30.0
12.0
205.0
19.0
63.0
252.0
254.0
18.0
80.0
227.0
30.0
12.0
205.0
19.0
63.0
252.0


c:\Users\roumba\Documents\Software\deep-events\.env\lib\site-packages\tifffile\tifffile.py:2001: UserWarning: <tifffile.TiffWriter 'image_221123_U2O…ma8_0gauss.tiff'> writing zero-size array to nonconformant TIFF
  warnings.warn(


254.0
18.0
80.0
227.0
sigma: 9
30.0
12.0
205.0
19.0
63.0
252.0
254.0
18.0
80.0
227.0
30.0
12.0
205.0
19.0


c:\Users\roumba\Documents\Software\deep-events\.env\lib\site-packages\tifffile\tifffile.py:2001: UserWarning: <tifffile.TiffWriter 'image_221123_U2O…ma9_0gauss.tiff'> writing zero-size array to nonconformant TIFF
  warnings.warn(


63.0
252.0
254.0
18.0
80.0
227.0
file number: 3
sigma: 1
88.0
164.0
116.0
137.0
86.0
55.0
88.0
164.0
116.0
137.0
86.0
55.0
sigma: 2
88.0
164.0
116.0
137.0
86.0
55.0
88.0
164.0
116.0
137.0
86.0
55.0
sigma: 3
88.0
164.0
116.0
137.0
86.0
55.0
88.0
164.0
116.0
137.0
86.0
55.0
sigma: 4
88.0
164.0
116.0
137.0
86.0
55.0
88.0
164.0
116.0
137.0
86.0
55.0
sigma: 5
88.0
164.0
116.0
137.0
86.0
55.0
88.0
164.0
116.0
137.0
86.0
55.0
sigma: 6
88.0
164.0
116.0
137.0
86.0
55.0
88.0
164.0
116.0
137.0
86.0
55.0
sigma: 7
88.0
164.0
116.0
137.0
86.0
55.0
88.0
164.0
116.0
137.0
86.0
55.0
sigma: 8
88.0
164.0
116.0
137.0
86.0
55.0
88.0
164.0
116.0
137.0
86.0
55.0
sigma: 9
88.0
164.0
116.0
137.0
86.0
55.0
88.0
164.0
116.0
137.0
86.0
55.0
file number: 4
sigma: 1
41.0
117.0
18.0
41.0
117.0
18.0
sigma: 2
41.0
117.0
18.0
41.0
117.0
18.0
sigma: 3
41.0
117.0
18.0
41.0
117.0
18.0
sigma: 4
41.0
117.0
18.0
41.0
117.0
18.0
sigma: 5
41.0
117.0
18.0
41.0
117.0
18.0
sigma: 6
41.0
117.0
18.0
41.0
117.0
18.0
sigma: 7
41.0
11

## Cut Negative Frames ##

In [6]:
folder_name=images_dir+'_neg'
filepath= os.path.join(training_dir , folder_name)
from myfunctions import image_crop_negative

for x in range(max_number):
    csv=datacsv[x]
    im=img[x]
    print('file number:',x)
    out_name=f'{output_name[x]}'

    list1=event_separation(csv)
    l=len(list1)
    image_crop_negative(l,list1, csv, im, out_name,filepath)

file number: 0
   axis-0      axis-1      axis-2
0    56.0  633.462488  832.188509
1    55.0  634.153176  832.015837
2    54.0  634.383406  831.382706
3    53.0  634.440963  831.555378
4    52.0  634.325848  832.648968
    axis-0      axis-1      axis-2
5     60.0  702.102110  774.473438
6     59.0  702.048989  774.154715
7     58.0  702.102110  775.429606
8     57.0  702.208351  776.492016
9     56.0  702.792676  776.438896
10    55.0  702.367712  775.323365
11    54.0  702.048989  774.898401
12    53.0  702.155230  774.526558
13    52.0  702.527074  775.588968
14    51.0  702.845796  775.164004
15    50.0  702.420833  775.429606
    axis-0      axis-1      axis-2
16    26.0  699.582358  736.381632
17    25.0  700.195849  735.359147
18    23.0  699.780587  736.301215
19    22.0  699.204969  736.946287
file number: 1
   axis-0       axis-1       axis-2
0   133.0  1267.294395  1929.843571
1   132.0  1268.532745  1930.034086
   axis-0       axis-1       axis-2
2    39.0  1363.430269  194